# TFM - Obtención de varias CNNs
## Ismael Franco Hernando
#### Imports

In [3]:
from tensorflow import keras
from mysql.connector import (connection)
import pandas as pd
import numpy as np
from tensorflow.keras.models import Sequential;
from tensorflow.keras.layers import Dense, Conv1D, MaxPooling1D, Flatten
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from imblearn.over_sampling import RandomOverSampler

#### Se realiza una conexión con la base de datos y se cargan las features de cada bobina

In [4]:
try:
    cnx = connection.MySQLConnection(user='leer', 
                                     password='candanal',
                                     host='138.100.82.178',
                                     database='deepquality_tmp')

except mysql.connector.Error as err:
    if err.errno == errorcode.ER_ACCESS_DENIED_ERROR:
        print("Usuario o contraseña incorrecto")
    elif err.errno == errorcode.ER_BAD_DB_ERROR:
        print("La base de datos no existe")
    else:
        print(err)

In [5]:
# Guardamos y mostramos las bobinas 
q01 = "SELECT * FROM FEATURES_1D"
fea1D = pd.read_sql(q01,cnx)
fea1D

,COILID,MID,ZNMAX_FAILURES,ZNMIN_FAILURES,CALIBRATED,TOTAL_TILEID,L_DIS,R_DIS,MAP,DECISSION_OP
0,225216688,123.0,0,6,0,26,13,12,"[-1, -1, -1, -1, -1, -1, 0, -1, -1, -1, -1, -1...",OK
1,225216688,124.0,4,0,0,26,3,6,"[1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",OK
2,225216688,201.0,1,1,0,26,3,4,"[-1, -1, -1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",OK
3,225216688,202.0,1,1,0,26,13,13,"[1, 1, 1, -1, -1, -1, -1, -1, -1, -1, -1, -1, ...",OK
4,225220725,123.0,2,5,0,54,17,27,"[1, 1, 1, 1, 0, 1, 0, 0, 0, 0, -1, 0, -1, 0, -...",OK
...,...,...,...,...,...,...,...,...,...,...
4635,233652453,202.0,2,0,0,16,8,8,"[0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1]",OK
4636,233667056,123.0,1,0,0,26,13,13,"[0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",OK
4637,233667056,124.0,1,1,0,26,13,13,"[0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, ...",OK
4638,233667056,201.0,1,0,0,26,13,13,"[0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, ...",OK


In [6]:
# Se cierra la conexión con la base de datos
cnx.close()

#### Union de los mapas
Para cada bobina se unen los mapas de la capa de arriba, sensores 123 y 124, y los mapas de las capas de abajo, sensores 201 y 202.

In [7]:
def unionMapas(datos):
    bobinas = datos['COILID'].unique()
    sensores = [(123.0, 124.0), (201.0, 202.0)]
    X = np.zeros(2320).tolist()
    Y = []
    contadorID = 0
    
    for bob in bobinas:
        for s in sensores:
            mapa1 = datos.loc[datos['COILID']==bob].loc[datos['MID']==s[0]].MAP.iloc[0]      
            mapa1 = np.array(mapa1[1:-1].split(',')).astype(int)
            mapa2 = datos.loc[datos['COILID']==bob].loc[datos['MID']==s[1]].MAP.iloc[0]      
            mapa2 = np.array(mapa2[1:-1].split(',')).astype(int)
            X[contadorID] = np.concatenate((mapa1, mapa2), axis=0)
            Y.append(datos.loc[datos['COILID']==bob].DECISSION_OP.iloc[0])
            contadorID+=1
            
    return X,Y      


In [8]:
X,Y = unionMapas(fea1D)

#### Procesado de los datos para usarlos en la CNN
En primer lugar se obtiene el mapa de mayor tamaño de entre todas las bobinas para hacer un padding y que todas las bobinas cuenten con el mismo tamaño.

In [9]:
longitudes = []
for x in X:
    longitudes.append(len(x))
    
maxLongitud = max(longitudes)

In [10]:
nuevoX = []
for x in X:
    nuevoX.append(np.pad(x, (0,maxLongitud-len(x)), 'constant', constant_values= 0))
    

A continuación, se expande la dimensión del array de 2D a 3D ya que la CNN de tensroflow necesita que los datos de entrada estén en 3D

In [11]:
X = np.expand_dims(nuevoX, axis=-1)
X.shape

(2320, 208, 1)

La CNN necesita que las clases estén en valor numérico, por lo que se pasa la label OK a 0 y NOK a 1

In [12]:
nuevoY = []
for y in Y:
    if y == 'OK':
        nuevoY.append(0)
    else:
        nuevoY.append(1) 
        
Y = nuevoY

#### División de los datos para entrenar y para test

In [13]:
Xtrain, Xtest, Ytrain, Ytest = train_test_split(X, Y, test_size=0.3, random_state=18)

#### Creación y evaluación del modelo

In [14]:
modelo = Sequential();
modelo.add(Conv1D(32, 3, activation='relu', input_shape=(maxLongitud, 1)))
modelo.add(MaxPooling1D(2))
modelo.add(Flatten())
modelo.add(Dense(32, activation='relu'))
modelo.add(Dense(8, activation='relu'))
modelo.add(Dense(1, activation='relu'))

modelo.summary()

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d (Conv1D)              (None, 206, 32)           128       
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 103, 32)           0         
_________________________________________________________________
flatten (Flatten)            (None, 3296)              0         
_________________________________________________________________
dense (Dense)                (None, 32)                105504    
_________________________________________________________________
dense_1 (Dense)              (None, 8)                 264       
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 9         
Tota

In [15]:
modelo.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [16]:
modelo.fit(Xtrain, Ytrain, epochs=10, batch_size=32)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Epoch 1/10
1624/1624 [==============================] - 0s 184us/sample - loss: 0.5991 - acc: 0.7161
Epoch 2/10
1624/1624 [==============================] - 0s 105us/sample - loss: 0.5379 - acc: 0.7580
Epoch 3/10
1624/1624 [==============================] - 0s 101us/sample - loss: 0.5081 - acc: 0.7543
Epoch 4/10
1624/1624 [==============================] - 0s 136us/sample - loss: 0.4974 - acc: 0.7722
Epoch 5/10
1624/1624 [==============================] - 0s 199us/sample - loss: 0.4681 - acc: 0.7857
Epoch 6/10
1624/1624 [==============================] - 0s 202us/sample - loss: 0.4618 - acc: 0.7863
Epoch 7/10
1624/1624 [==============================] - 0s 206us/sample - loss: 0.4049 - acc: 0.8030
Epoch 8/10
1624/1624 [==============================] - 0s 189us/sample - loss: 0.4051 - acc: 0.8122
Epoch 9/10
1624/1624 [==============================] - 0s 195us/sample - loss: 0.3594 - acc: 0.82

In [17]:
# Se evalua el modelo
resultado  = modelo.evaluate(Xtest,  Ytest)
print("\nEl test loss obtenido ha sido:", resultado[0])
print("La precisión obtenida ha sido:", resultado[1])

696/696 [==============================] - 0s 231us/sample - loss: 1.0126 - acc: 0.7356

El test loss obtenido ha sido: 1.0126329742628952
La precisión obtenida ha sido: 0.7356322


In [18]:
# Se realizan predicciones
Ypredic= modelo.predict(Xtest)

# Se pasan las predicciones a 0s y 1s
Yprocesada = []
for y in Ypredic:
    if y <= 0.5:
        Yprocesada.append(0)
    else:
        Yprocesada.append(1)

In [19]:
# Se muestra la matriz de confusión
print("La matriz de confusion obtenida es:\n", confusion_matrix(Ytest, Yprocesada))

La matriz de confusion obtenida es:
 [[425  82]
 [102  87]]


#### Sobremuestreo de los datos

In [20]:
# Se crea el ROS
ros = RandomOverSampler()

In [21]:
# Se realiza un sobremuestreo de los datos
Xros, Yros = ros.fit_resample(nuevoX, Y)

In [22]:
# Se pasan los datos 2D a 3D para el modelo
Xros = np.expand_dims(Xros, axis=-1)
Xros.shape

(3340, 208, 1)

In [23]:
# Se obtienen los datos para train y para test
Xtrainr, Xtestr, Ytrainr, Ytestr = train_test_split(Xros, Yros, test_size=0.3, random_state=18)

In [24]:
# Se crea y se muestra el modelo
modelo2 = Sequential();
modelo2.add(Conv1D(32, 3, activation='relu', input_shape=(maxLongitud, 1)))
modelo2.add(MaxPooling1D(2))
modelo2.add(Flatten())
modelo2.add(Dense(32, activation='relu'))
modelo2.add(Dense(8, activation='relu'))
modelo2.add(Dense(1, activation='relu'))

modelo2.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_1 (Conv1D)            (None, 206, 32)           128       
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 103, 32)           0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 3296)              0         
_________________________________________________________________
dense_3 (Dense)              (None, 32)                105504    
_________________________________________________________________
dense_4 (Dense)              (None, 8)                 264       
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 9         
Total params: 105,905
Trainable params: 105,905
Non-trainable params: 0
________________________________________________

In [25]:
# Se compila el modelo
modelo2.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [26]:
# Se entrena el modelo
modelo2.fit(Xtrainr, Ytrainr, epochs=10, batch_size=32)

Epoch 1/10
2338/2338 [==============================] - 0s 203us/sample - loss: 0.7566 - acc: 0.6091
Epoch 2/10
2338/2338 [==============================] - 1s 243us/sample - loss: 0.6457 - acc: 0.6775
Epoch 3/10
2338/2338 [==============================] - 1s 265us/sample - loss: 0.5769 - acc: 0.7083
Epoch 4/10
2338/2338 [==============================] - 1s 238us/sample - loss: 0.6184 - acc: 0.6788
Epoch 5/10
2338/2338 [==============================] - 1s 234us/sample - loss: 0.6338 - acc: 0.6719
Epoch 6/10
2338/2338 [==============================] - 1s 222us/sample - loss: 0.5340 - acc: 0.7335
Epoch 7/10
2338/2338 [==============================] - 0s 210us/sample - loss: 0.5018 - acc: 0.7609
Epoch 8/10
2338/2338 [==============================] - 1s 238us/sample - loss: 0.4627 - acc: 0.7789
Epoch 9/10
2338/2338 [==============================] - 1s 231us/sample - loss: 0.4426 - acc: 0.7908
Epoch 10/10
2338/2338 [==============================] - 1s 253us/sample - loss: 0.4149 - a

In [27]:
# Se evalua el modelo
resultado  = modelo2.evaluate(Xtestr,  Ytestr)
print("\nEl test loss obtenido ha sido:", resultado[0])
print("La precisión obtenida ha sido:", resultado[1])

1002/1002 [==============================] - 0s 177us/sample - loss: 0.6907 - acc: 0.7255

El test loss obtenido ha sido: 0.6907426525732714
La precisión obtenida ha sido: 0.7255489


In [28]:
# Se realizan predicciones
Ypredic= modelo.predict(Xtestr)

# Se pasan las predicciones a 0s y 1s
Yprocesada = []
for y in Ypredic:
    if y <= 0.5:
        Yprocesada.append(0)
    else:
        Yprocesada.append(1)

In [29]:
# Se muestra la matriz de confusión
print("La matriz de confusion obtenida es:\n", confusion_matrix(Ytestr, Yprocesada))

La matriz de confusion obtenida es:
 [[424  52]
 [189 337]]
